# Multi-Armed Bandit 

Multi-Armed Bandits are a type of Reinforcement Learning algorithmn that is used to learn the best actions to maximize an expected gain. The algorithm learns about the distribution of the gain for the different actions over time by using an exploration(aquire new knowledge and exploitation(optimize the decisions based on existing knowledge) tradeoff stragegy. The agent attempts to balance these competing tasks in order to maximize the total gain over time.


## Epsilon Greedy
$\epsilon$ = probability of exploration.  This is the is the percentage of exploration actions the agent will take. For example if $\epsilon$=0.05, then 5% of actions will be exploration, and 95% will be exploitation. 

Epsilon Greedy allows the agent to explore all actions. The agent learns the best actions by updating the mean of each bandit after each action is taken. 

After the optimal actions are identified, the epsilon greedy algorithm will still explore suboptimal actions at the rate of $\epsilon$.

## Optimistic Initialization
Optimistic intialization is an alternative to the epsilon greedy algorithm. It forces the agent to explore all actions by setting the inital mean of all the bandits as very high, ie an upper limit. Means should be selected to be much higher than what the true mean could be. This forces the agent to explore all the bandits, and the bandit's means will decrease quickly. 

## UCB1
Similar to optimistic initalization, UCB1 initializes the mean of the bandits using an upper bound. This upper bound is based on confidence bounds and is determined by the Hoeffding’s Inequality.
$$P\{|\bar{X}-\mu| \geq \epsilon\} \leq 2e^{-2\epsilon^2N}$$
which rearranges to an upper bound on the mean for the jth bandit
$$X_{UCB-j}=\bar{X_j}+\sqrt{2\frac{lnN}{N_j}}$$
Where N is the number of times all bandits have been played. 

This allows the initialized mean values of the bandits to shrink as the agent tries each bandit, and becomes more confident in our estimate of the bandits true mean. 

## Bayesian Multi-Armed Bandits

In the bayesian multiarmed bandit, the an upper limit is used to initalize the means of the bandits. The upper limit is a upper confidence bound calculated using a distribution of the mean given the observed data, $p(\mu|X)$. This is the posterior of $\mu$.

We use bayes rule to calculate the posterior. 

$$p(\mu|X)=p(X|\mu)p(\mu)$$

We assume X follow a normal $X \sim Normal(\mu, \sigma^2/N)$.

And we put a prior on the mean $\mu \sim Beta(a, b)$

The upper limit is selected as the max of the samples from each of the bandits. The distribution is fat )has large confidence intervals) when few samples have been observed, and becomes skinnier as we approximate the true mean. 

In [ ]:
from src.reinforcement_learning import GreedyBandit, OptimisticBandit, UCB1Bandit, BayesianBandit

In [ ]:
bandits = GreedyBandit.perform_epsilon_greedy(bandit_true_means=[0.01, 0.05, 0.5], N=1000, epsilon=0.1)
for b in bandits: print(b.mean)   

In [ ]:
bandits = OptimisticBandit.perform_optimistic_initialization(bandit_true_means=[0.01, 0.05, 0.5], N=1000, upper_limit=10)
for b in bandits: print(b.mean)

In [ ]:
bandits = UCB1Bandit.perform_ucb1(bandit_true_means=[0.01, 0.05, 0.5], N=1000, upper_limit=10)
for b in bandits: print(b.mean)

In [ ]:
bandits = BayesianBandit.perform_bayesian_bandits(bandit_true_means=[0.01, 0.05, 0.5], N=1000)
for b in bandits: print(b.mean)

# Contexual Multi-Armed Bandits for Recommender System

Contexual Multi-Armed Bandits use the MAB framework to recommend items to users. Actions are decided based on the expected reward, and based on a state learned from the user's data. As a result, the recommendations are personalized at the user level. 

"The algorithm observes a context, makes a decision, choosing one action from a number of alternative actions, and observes an outcome of that decision. An outcome defines a reward. The goal is to maximize average reward."

Why this and not a traditional ML algorithm that retrains when it gets new info?
* cause of cold start. It makes informed neighborhood recs that optimize the probability they will enage with eh the item quickly, and give us engagemetn sthat will segment.
* Not cause of live learning. i mean you get taht from retraining it would update with new info
* cause of explore exploit tradeoff? Np. a trained ML algo would still explore in a sense it would recoomemdn things we _think_ they would like, but when retrain we encorporate what we learned from taht smart exploration. 
* So why then? Just cold start? Yeah. Both rec cold start and frequency are same problem. No data on the user yet. 

A particularly useful version of the multi-armed bandit is the contextual multi-armed bandit problem. In this problem, in each iteration an agent has to choose between arms. Before making the choice, the agent sees a d-dimensional feature vector (context vector), associated with the current iteration. The learner uses these context vectors along with the rewards of the arms played in the past to make the choice of the arm to play in the current iteration. Over time, the learner's aim is to collect enough information about how the context vectors and rewards relate to each other, so that it can predict the next best arm to play by looking at the feature vectors


Can I crete a bayesian network/graphical model similar to VAE to model this?

I want to use MAB to solve 2 challenges in recommender systems:
1. The cold start problem. 

Recommend items to new users in a way that will optimize our chances of them engaging with the item. Goal is to collect interaction data from these new users so that we can train a ML recommender. The items shown to users should have a high change of being interacted with, while still informing us about the kind of user they are, meaning the items should represent distinct clusters of users. =  The baselien for this is popularity recs, and it is expected to outperform because the items shown can segment users instead of jsut get interactions.  

2. Make frequency of contact recommendations that drive engagement in the environment where there is not enough variance in historical data to train an ML model, and more items will always increase cliks, but cause user fautifue resulting in unsubscribes. This leaverages the explor exploit tradeoff that MAB's offer, while solving the existing cold start problem. 

For frequency optimization, frequency has not changed at all. Meaning each user has only seem one item. The goal is to use the MAB to vary frequency in real time while optimizing the tradeoff between engagements and dissengagements. 

The MAB balances the tradeoff between exploitation and exploration, which will allow us to safely vary frequency enough to learn the right policy, while maximizing the customers probability of engaging. 



In [ ]:
import pandas as pd
books = pd.read_csv('data/book_ratings.csv', sep=";", escapechar='\\', encoding='CP1252', low_memory=False)
books.columns = ["user", "isbn", "rating"]
books.head(3)

In [ ]:
top_10_books = books.isbn.value_counts().index[:10].tolist()
books = books.query('isbn in @top_10_books')
print("{} unique users".format(books.user.nunique()))
print("{} unique books".format(books.isbn.nunique()))

In [ ]:
books.head()